In [11]:
import numpy as np
from spectral_cube import SpectralCube
from astropy.io import fits
from reproject import reproject_interp

In [27]:
def get_cube(cube_filename):
    cube = SpectralCube.read(cube_filename)
    cube.allow_huge_operations = True
    cube = cube.to('K')
    return(cube)

def reproject_index_map(cube, index_map_hdu):
        
    # Create 2D header
    header_2d = cube.hdu.header.copy()
    del header_2d['*3*']
    header_2d['NAXIS'] = header_2d['WCSAXES'] = 2 
    index_map_data, _ = reproject_interp(index_map_hdu, header_2d, order='nearest-neighbor')
    
    return(index_map_data)

def average_spectrum_for_indexes(cube, index_map):
    
    # Load the data cube
    unique_indexes = np.unique(index_map)
    spectra_dict = {}

    for index in unique_indexes:
        # Create a mask for the current index
        mask = (index_map == index)

        # If the cube is not loaded in memory, the following approach is more memory efficient
        masked_cube = cube.with_mask(mask[:, :, np.newaxis])

        # Calculate the mean spectrum for the masked region
        mean_spectrum = masked_cube.mean(axis=(1, 2))

        spectra_dict[index] = mean_spectrum

    return spectra_dict

def fit_gaussian(spectrum):
    sp = pyspeckit.Spectrum(data=spectrum.value, xarr=spectrum.spectral_axis.value)
    sp.specfit.fittype = 'gaussian'
    sp.specfit(guesses=[spectrum.max().value, spectrum.spectral_axis[spectrum.argmax()].value, 1.0])
    return sp.specfit.parinfo

In [24]:
# Define files 
index_map_filename = "/Users/abarnes/Dropbox/work/scratch/ACES1/data/12m_cont/cont_12m.fits"
cube_filename = "/Users/abarnes/Dropbox/work/scratch/ACES1/data/12m7mtp_lowres/HNCO_7m12mTP_CubeMosaic_downsampled9_downsampledspectrally.fits"

#Load cube
cube = get_cube(cube_filename)

#Regrid index map 
index_map_hdu = fits.open(index_map_filename)[0]
index_map = reproject_index_map(cube, index_map_hdu)

# Get spectrum 
average_spectra = average_spectrum_for_indexes(cube, index_map)

# Fit Gaussians and compile results
results = []
for index, spectrum in average_spectra.items():
    fit = fit_gaussian(spectrum)
    amplitude, center, width = fit[0].value, fit[1].value, fit[2].value
    results.append((index, amplitude, center, width))

# Convert results to Astropy table
table = Table(rows=results, names=("Index", "Amplitude", "Center", "Width"))
print(table)